# Data Science блог с помощью Fastpages
> Компиляция статей и документации по `fastpages` с переводом на русский и моим субъективным мнением.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, theory]

В конце февраля 2020 года ребята из `fast.ai` представили миру `fastpages` - платформу для ведения блога.

Сразу скажу, что `fastpages` основан на `Jekyll`, о котором на Хабре есть множество постов.

Главное отличительная черта и приемущество `fastpages` состоит в поддерживаемых из коробки форматах постов:
- Jupyter ноутбуки (расширение `.ipynb`);
- Markdown файлы (расширение `.md`);
- Word файлы (расширение `.docx`)

Таким образом, автору блога необходимо сохранить пост в любом из перечисланных выше форматах в соответствующей директории:
- './_notebooks/' для `.ipynb`;
- './_posts/' для `.md`;
- './_word/' для `.docx`.

А все остальное сделает `fastpages`, как утверждают его авторы.
`fastpages` использует Github Pages для хостинга и Github Actions для автоматизации публикации постов.

Таким образом `fastpages` является доработкой связки Github Pages + `Jekyll`, где можно сразу же из Jupyter ноутбука получить опубликованный пост.

## Первичная настройка `fastpages`
Если хотите самостоятельно разобраться, то вот [официальная инструкция](https://github.com/fastai/fastpages#setup-instructions) по настройке в репозитории `fastpages`.

Процесс настройки `fastpages`:

1. Создать собственную копию репозитория из шаблона `fastpages` по [ссылке](https://github.com/fastai/fastpages/generate)
![](../images/ds-blog-fastpages/new-repo-fastpages.png)
2. Далее автоматически откроется pull request (через ~ 30 секунд), который отвечает за настройку вашего блога, чтобы он мог начать работать.
![](../images/ds-blog-fastpages/init-setup.png)
3. Вам нужно выполнить иструкции из полученного pull request'a и вы получите свою собственную уже работающую платформу для блога.

Также есть [видео туториал](https://youtu.be/L0boq3zqazI) с пошаговой настройкой.

## `fast_template` - младший брат `fastpages`
Стоит упомянуть, что ранее `fast.ai` выпустили аналогичный проект под названием `fast_template`, который еще проще в настройке, но не поддерживает автоматическое создание постов из Word и Jupyter файлов, а также многие другие функции перечисленные выше. Поскольку `fastpages` более гибок и расширяем, его авторы рекомендуют использовать его там, где это возможно.

Авторы предполагают, что `fast_template` может быть лучшим вариантом для тех, кто ведет не технические блоги. В этом случае посты можно создавать только с помощью встроенного онлайн-редактора Github, не заморачиваясь с использованием `git`.

## Jupyter Notebooks & Fastpages
`fastpages` из коробки поддерживает различные удобства.

### Options via FrontMatter
Первая ячейка в вашем Jupyter ноутбуке (а также первые строки в Markdown файлах) содержит метаданные, которые могут включать/выключать опции связанные с постом.

```
# Title
> Awesome summary

- toc:true- branch: master
- badges: true
- comments: true
- author: Hamel Husain & Jeremy Howard
- categories: [fastpages, jupyter]
```

- `toc` - при значении `true` автоматически будет сгенерировано оглавление поста;
- `badges` - при значении `true` отображаются ссылки `Google Colab` и `GitHub`;
- `comments` - при значении `true` будут включены комментарии ([больше деталей](https://github.com/fastai/fastpages#enabling-comments));
- `author` - при значении `true` отображаются имена авторов;
- `categories` - позволяют группировать посты по тегам (на странице "Tags").

Для указания таких в Markdown файлах необходимо в начале файла задать опции как и в ноутбуке, только поместив эти метаданные между строк содержащих по три минуса, т.е. `---`.

Выглядит это так:

```
---
toc: true
layout: post
description: Awesome description.
categories: [markdown]
title: Title
---
```


### Сворачивание/скрытие кода
TODO: выяснить все про `#collapse`

Приятной функциональностью этого движка для блога является возможность скрывать код и/или результаты его выполнения. Это позволяет не нагружать посты отображением простыни кода или огромного количество принтов (что бывает при обучении нейросетей по эпохам), скрывая эти большие по размеру элементы, но не выкидывая их из поста на совсем.

Поместив флаг `#collapse-hide`, в первую строку любой ячейки, вы скроете код этой ячейки внутри поста. Но в замен появится кнопка, позволяющая показать эту ячейку.

In [1]:
#collapse-hide
import altair as alt
import pandas as pd
import numpy as np

Флаг `#collapse-show` позволяет показать ячейку по умолчанию, но дает читателю возможность скрыть ее.

In [2]:
#collapse-show
np.random.seed(42)
source = pd.DataFrame(
    np.cumsum(np.random.randn(100, 3), 0).round(2),
    columns=['A', 'B', 'C'],
    index=pd.RangeIndex(100, name='x')
)
source = source.reset_index().melt('x', var_name='category', value_name='y')

Чтобы полностью скрыть ячейки (а не просто свернуть их), смотрите [сюда](https://github.com/fastai/fastpages#hide-inputoutput-cells).

### Интерактивные графики с помощью [Altair](https://altair-viz.github.io/)

In [3]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='x:Q',
    y='y:Q',
    color='category:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='x:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'y:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='x:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)

### Отображение таблиц

In [4]:
source

,x,category,y
0,0,A,0.50
1,1,A,2.02
2,2,A,3.60
3,3,A,4.14
4,4,A,4.38
...,...,...,...
295,95,C,5.31
296,96,C,5.10
297,97,C,5.95
298,98,C,6.85


## Другие возможности ``

## Pros&Cons
Плюсы:
- отсутствие сторонней рекламы;
- ;

Минусы:
- непонятно, как сделать структуру
- латиница в git : "_posts/2020-05-26-DS \320\221\320\273\320\276\320\263 \321\201 \320\277\320\276\320\274\320\276\321\210\321\214\321\216 Fastpages.md";

## DS/ML блоги
- [Анализ малых данных](https://dyakonov.org/) блог Александра Дьяконова;
- Andrej Karpathy [github.io](http://karpathy.github.io/), [medium](https://medium.com/@karpathy);
- [Machine Learning Mastery](https://machinelearningmastery.com/blog/) by Jason Brownlee;

### Блоги компаний
- [Fast.ai](https://www.fast.ai/) + [fastpages blog](https://fastpages.fast.ai/);
- [Airbnb](https://medium.com/airbnb-engineering/ai/home);
- [Uber](https://eng.uber.com/category/articles/ai/);
- [OpenAI](https://openai.com/blog/);
- [DeepMind](https://deepmind.com/blog);
- [Nvidia](https://blogs.nvidia.com/blog/category/deep-learning/) + [AI podcast](https://blogs.nvidia.com/ai-podcast/);
- Microsoft [AI blog](https://blogs.microsoft.com/ai/) + [ML devblogs](https://devblogs.microsoft.com/cse/tag/,machine-learning-ml/)

## Полезные ссылки:
- [Репозиторий](https://github.com/fastai/fastpages#setup-instructions) проекта `fastpages`;
- [Introducing fastpages](https://fastpages.fast.ai/fastpages/jupyter/2020/02/21/introducing-fastpages.html);